In [ ]:
import os
import yaml
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from ultralytics.utils.ops import segments2boxes
    
class YOLOSegmentationVisualizer:
    def __init__(self, yaml_path):
        # Load class names from yaml file
        with open(yaml_path, 'r') as f:
            self.yaml_data = yaml.safe_load(f)
            self.class_names = self.yaml_data['names']
    
    def read_yolo_label(self, label_path):
        segments = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    data = line.strip().split()
                    class_id = int(data[0])
                    # Extract polygon points
                    points = [float(x) for x in data[1:]]
                    # Reshape points into pairs of x,y coordinates
                    polygon = np.array(points).reshape(-1, 2)
                    bbox = segments2boxes(polygon.reshape(1,-1, 2))
                    segments.append([class_id, polygon, bbox])
                    #new_polygon = remove_zero_area_points(polygon)
                    #new_bbox = segments2boxes(new_polygon.reshape(1,-1, 2))
                    #segments.append([class_id, polygon, bbox, new_polygon, new_bbox])
        return segments

    def plot_image_with_segments(self, image_path, label_path):
        # Read image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        height, width = image.shape[:2]
        
        # Read labels
        segments = self.read_yolo_label(label_path)
        
        # Create figure and axes
        fig, ax = plt.subplots(1, figsize=(12, 8))
        ax.imshow(image)
        
        # Generate random colors for each class
        np.random.seed(42)
        colors = np.random.rand(len(self.class_names), 3)
        
        # Plot each segment
        for i,segment in enumerate(segments):
            class_id, polygon, bbox = segment #, new_polygon, nbbox
            
            # Convert normalized coordinates to pixel coordinates
            polygon[:, 0] *= width
            polygon[:, 1] *= height
            #new_polygon[:, 0] *= width
            #new_polygon[:, 1] *= height
            #print(class_id)
            #if class_id != 4:
            #    continue
            
            # Create polygon patch
            polygon_patch = plt.Polygon(polygon, 
                                      fill=True, 
                                      alpha=0.4, 
                                      color=colors[class_id],
                                      linewidth=2)
            c = ((bbox[0, 0] - bbox[0,2]/2) * width, (bbox[0, 1] - bbox[0,3] /2) * height)
            w = bbox[0, 2] * width
            h = bbox[0, 3] * height
            bbox_patch = plt.Rectangle(c, w, h, linewidth=1, fill=False, color=colors[class_id])
            ax.add_patch(bbox_patch)
            #c = ((nbbox[0, 0] - nbbox[0,2]/2) * width, (nbbox[0, 1] - nbbox[0,3] /2) * height)
            #w = nbbox[0, 2] * width
            #h = nbbox[0, 3] * height
            #nbbox_patch = plt.Rectangle(c, w, h, linewidth=2, fill=False, color=colors[class_id])
            #ax.add_patch(nbbox_patch)

            ax.add_patch(polygon_patch)
            
            # Add label
            class_name = self.class_names[class_id]
            centroid = np.mean(polygon, axis=0)
            ax.text(centroid[0], centroid[1], class_name + str(i), 
                   color='white', 
                   fontsize=10, 
                   bbox=dict(facecolor=colors[class_id], alpha=0.8))
        
        plt.axis('off')
        plt.show()
        
    def visualize_single_image(self, image_path, label_path=None):
        """
        Visualize a single image with its segmentation mask
        Args:
            image_path: Path to the image file
            label_path: Path to the label file. If None, will try to find the corresponding label file
        """
        if label_path is None:
            # Try to find the corresponding label file
            image_dir = os.path.dirname(image_path)
            image_name = os.path.splitext(os.path.basename(image_path))[0]
            label_dir = os.path.join(os.path.dirname(os.path.dirname(image_dir)), 'labels')
            label_path = os.path.join(label_dir, image_name + '.txt')
        
        print(f"Visualizing: {os.path.basename(image_path)}")
        self.plot_image_with_segments(image_path, label_path)

    def visualize_batch(self, images_dir, labels_dir, num_images=5):
        """
        Visualize a batch of images with their segmentation masks
        """
        image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
        polygon
        for image_file in image_files[:num_images]:
            image_path = os.path.join(images_dir, image_file)
            label_path = os.path.join(labels_dir, os.path.splitext(image_file)[0] + '.txt')
            
            print(f"Visualizing: {image_file}")
            self.plot_image_with_segments(image_path, label_path)


In [ ]:
visualizer = YOLOSegmentationVisualizer(yaml_path)

base_path = "/media/charles/DATA/Programs/datasetManipulation/datasets/Dataset-ViPARE-22/"
img_name = "20240507_165357-Skylex_jpg.rf.8de65629485d883ae0e311accc7e517b"
image_path = base_path + "test/images/" + img_name + ".jpg"
label_path = base_path + "test/labels/" + img_name + ".txt"
visualizer.visualize_single_image(image_path, label_path)

In [ ]:
img_idx = 1
base_path = "/media/charles/DATA/Programs/datasetManipulation/datasets/Dataset-ViPARE-22-split/"
img_name = "IMG_8659_JPG.rf.0a691e72c9860e3a0541254de9b3202a_"+str(img_idx)
img_name = "20240507_165357-Skylex_jpg.rf.8de65629485d883ae0e311accc7e517b_"+str(img_idx)
image_path = base_path + "test/images/" + img_name + ".jpg"
label_path = base_path + "test/labels/" + img_name + ".txt"
yaml_path = base_path + "data.yaml"
visualizer = YOLOSegmentationVisualizer(yaml_path)

visualizer.visualize_single_image(image_path, label_path)


In [ ]:
image = cv2.imread(image_path)

In [ ]:
/media/charles/DATA/Programs/datasetManipulation/datasets/Dataset-ViPARE-22-split/test/images/20240429_163624-Skylex_jpg.rf.3e3399025ede7328298c3e31d021758b_3.jpg